# Combine CSVs For Processing
Combines the CSVs so all relevant data is on one line and the data is placed based on linking keys through the different CSVs

Everything from this markdown cell to the next one is just a sanity check to make sure the chunking for the CSVs is not giving an odd output. Don't run unless you have some reason to check.

# Combine with Dask

We use dask because it automatically chunks data

We need to determine what features we want to be able to determine which CSVs we need to merge

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.types import LongType

In [2]:
# change to be reflective of your environment
data_dir = '/home/cole/Workspace/School/Capstone/data/first_data_set/TestData/'

In [3]:
# initiates a SparkContext which is necessary for accessing data in Spark
sc = SparkContext()
sqlContext = SQLContext(sc)
# change to match your environment
output_dir = data_dir + "/merge_data"

In [4]:
# based around linking to BACKUP_OBJECTS
CSV_Merge_List = {
    'NODES' : 'NODEID',
    'FILESPACES' : 'FSID',
    'AF_BITFILES' : ['OBJID', 'BFID'],
    'SD_RECON_ORDER' : 'OBJID',
    'SS_POOLS' : 'POOLID',
    'SD_CHUNK_LOCATIONS' : 'POOLID',}

In [5]:
AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/AF_BITFILES.csv"])
AFBF.count()

18048

In [9]:
BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])
BACKUP_OBJECTS.count()

5823871

In [6]:
AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/AF_BITFILES.csv"])
BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])
ARCHIVE_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/ARCHIVE_OBJECTS.csv"])
FILESPACES = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/FILESPACES.csv"])
NODES = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/NODES.csv"])
SD_CHUNK_COPIES = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/SD_CHUNK_COPIES.csv"])
SD_CHUNK_LOCATIONS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/SD_CHUNK_LOCATIONS.csv"])
SD_CONTAINERS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/SD_CONTAINERS.csv"])
SD_NON_DEDUP_LOCATIONS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/SD_NON_DEDUP_LOCATIONS.csv"])
SDRO = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SD_RECON_ORDER.csv'])
SS_POOLS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SS_POOLS.csv'])


In [12]:
BACKUP_OBJECTS.select(['OBJTYPE', 'BFSIZE', 'LOGICAL_SIZE', 'METADATASIZE', 'HDRSIZE', 'STG_HINT', 'FLAGS']).show(2000)

+-------+------+------------+------------+-------+--------+-----+
|OBJTYPE|BFSIZE|LOGICAL_SIZE|METADATASIZE|HDRSIZE|STG_HINT|FLAGS|
+-------+------+------------+------------+-------+--------+-----+
|      1|     0|        null|        null|      0|       1|    0|
|      1|     0|        null|        null|      0|       1|    0|
|      1|     0|        null|        null|      0|       1|    0|
|      1|     0|        null|        null|      0|       1|    0|
|      2| 14336|        null|           0|    323|       1|    0|
|      2|     0|        null|           0|      0|       1|    0|
|      2| 13312|        null|           0|    324|       1|    0|
|      2| 15360|        null|           0|    325|       1|    0|
|      2| 13312|        null|           0|    325|       1|    0|
|      2| 13312|        null|           0|    325|       1|    0|
|      2| 14336|        null|           0|    325|       1|    0|
|      2| 16384|        null|           0|    325|       1|    0|
|      2| 

In [13]:
objtype = BACKUP_OBJECTS.select(['OBJTYPE'])

In [14]:
objtype = [row['OBJTYPE'] for row in BACKUP_OBJECTS.select("OBJTYPE").collect()]

In [29]:
objtype.count('2') / len(objtype)

0.9658237622364918

In [28]:
objtype.count('1') / len(objtype)

0.03412833148261697

In [19]:
set(objtype)

{'1',
 '11',
 '12',
 '17',
 '18',
 '2',
 '20',
 '2016-10-11 19:37:05.000000',
 '2016-10-13 18:07:21.208052',
 '2016-10-13 18:07:21.216212',
 '2016-11-08 18:24:01.718047',
 '2016-11-08 18:24:01.730117'}

In [30]:
METADATASIZE = [row['METADATASIZE'] for row in BACKUP_OBJECTS.select("METADATASIZE").collect()]

In [31]:
set(METADATASIZE)

{'0',
 '100',
 '1000',
 '1002',
 '1004',
 '1006',
 '1008',
 '101',
 '1010',
 '1012',
 '1014',
 '1016',
 '1018',
 '102',
 '1020',
 '1022',
 '1024',
 '1026',
 '1028',
 '103',
 '1030',
 '1032',
 '1036',
 '1038',
 '104',
 '1040',
 '1042',
 '1044',
 '1046',
 '1048',
 '105',
 '1050',
 '1052',
 '1054',
 '1056',
 '106',
 '1060',
 '1062',
 '1064',
 '1066',
 '1068',
 '107',
 '1070',
 '1072',
 '1074',
 '1076',
 '1078',
 '108',
 '1080',
 '1082',
 '1084',
 '1086',
 '1088',
 '109',
 '1090',
 '1092',
 '1094',
 '1096',
 '1098',
 '110',
 '1100',
 '1102',
 '1104',
 '1106',
 '1108',
 '111',
 '1110',
 '1114',
 '1116',
 '1118',
 '1120',
 '1122',
 '1124',
 '1126',
 '1128',
 '113',
 '1130',
 '1132',
 '1134',
 '1136',
 '1138',
 '114',
 '1140',
 '1142',
 '1144',
 '1146',
 '1148',
 '115',
 '1150',
 '1152',
 '1154',
 '1156',
 '1158',
 '1160',
 '1162',
 '1164',
 '1166',
 '1168',
 '1170',
 '1172',
 '1174',
 '1176',
 '1178',
 '1180',
 '1182',
 '1184',
 '1186',
 '1188',
 '1190',
 '1192',
 '1194',
 '1196',
 '1198',
 

In [32]:
METADATASIZE.count('0') / len(METADATASIZE)

0.3034376963363371

In [36]:
meta_set = set(METADATASIZE)
for item in meta_set:
    percent = METADATASIZE.count(item) / len(METADATASIZE)
    if percent >= 0.02:
        print('{} : {}'.format(item, percent))

190 : 0.02681292219556374
90 : 0.03380706749857612
238 : 0.1631531673692635
95 : 0.11833263477161496
0 : 0.3034376963363371
194 : 0.295160383875261


In [37]:
STG_HINT = [row['STG_HINT'] for row in BACKUP_OBJECTS.select("STG_HINT").collect()]

In [40]:
meta_set = set(STG_HINT)
for item in meta_set:
    percent = STG_HINT.count(item) / len(STG_HINT)
    if percent >= 0.001:
        print('{} : {}'.format(item, percent))

2 : 0.3492754217941984
1 : 0.6507056904248051
None : 1.888778099652276e-05


In [41]:
FLAGS = [row['FLAGS'] for row in BACKUP_OBJECTS.select("FLAGS").collect()]

In [43]:
meta_set = set(FLAGS)
for item in meta_set:
    percent = FLAGS.count(item) / len(FLAGS)
    print('{} : {}'.format(item, percent))

8 : 0.25663978477545263
16 : 2.4897529495416363e-05
0 : 0.7410143528247792
None : 0.0023209648702727102


In [18]:
AFBF = AFBF.withColumn('POOLID', AFBF.POOLID.cast(IntegerType()))
SD_CONTAINERS = SD_CONTAINERS.withColumn('POOLID', SD_CONTAINERS.POOLID.cast(IntegerType()))
SD_CHUNK_LOCATIONS = SD_CHUNK_LOCATIONS.withColumn('CHUNKID', SD_CHUNK_LOCATIONS.CHUNKID.cast(LongType()))
SD_CHUNK_LOCATIONS = SD_CHUNK_LOCATIONS.withColumn('POOLID', SD_CHUNK_LOCATIONS.POOLID.cast(IntegerType()))

In [19]:
tape = [row['POOLID'] for row in AFBF.select("POOLID").distinct().collect()]
cloud = set([row['POOLID'] for row in SD_CONTAINERS.filter(SD_CONTAINERS.TYPE.rlike('3|4')).select('POOLID').distinct().collect()])
directory = set([row['POOLID'] for row in SD_CONTAINERS.filter(SD_CONTAINERS.TYPE.rlike('1|2')).select('POOLID').distinct().collect()]) - cloud

In [20]:
tape

[-1000000, -1, 6, -9, 72, 4, 82, 135, 42]

In [21]:
cloud

{24, 37, 38, 60, 71, 83, 95, 112, 114}

In [22]:
directory

{27, 138}

In [8]:
merge = BACKUP_OBJECTS.join(AFBF, BACKUP_OBJECTS['OBJID'] == AFBF['BFID'], how='left')
merge = merge.join(SDRO, ['OBJID'], how='left')

In [26]:
merge.count()

50482944

In [30]:
temp = NODES.withColumnRenamed('NODEID', 'tempNODEID')

In [9]:
df = merge.join(NODES, ['NODEID'])

In [11]:
# df.select(['OBJTYPE', 'BFSIZE', 'LOGICAL_SIZE', 'METADATASIZE', 'HDRSIZE', 'STG_HINT', 'CLNTTYPE']).show(2000)

AnalysisException: "cannot resolve '`CLNTTYPE`' given input columns: [PARENTHL, EXTOBJID, GUID_FORMAT, CLNTOSLEVEL, PLATFORM, RECOVER_DAMAGED, DECOMM_DATE, POOLID, METADATASIZE, PVENDOR, SEND_HDR, HL_NAME, SESSCANCELED, OBJCHGID, ATRISK_VALUE, MCID, SRVID, CRCREQUIRED, META_UPDATE, LOGICAL_SIZE, S3_ACCESS_ID, MAXNUMMP, ATTRLENGTH, CLIENT_HLA, BFID, ENCRYPT_KEY, AUTHENTICATION, ATRISK_TYPE, PWSETDATE, DEACDATE, OBJID, ROLE, FAILOVER_WRT_DELAY, PBRAND, BACKUPINIT, INSDATE, PMODEL, EXTTOKEN, CLNTPRODUCTS, MACADDR, CGID, FLAGS, RECOGTOKEN, SESSMEDIAWAIT, BYPASSRECOG, SESSCOMMWAIT, BYTES_STORED, GMTINSDATE, TGTVERSION, BACKDEL, SESSCOMMMETH, GUID_HLA, NODESTATE, PFILES_STORED, REPL_SPRULE, ATRISK_INTERVAL, BFSIZE, CLNTRELEASE, PWRESET, GROUPID, CLNTVERSION, APPVERSION, AGGR_ESTSIZE, OBJINFO, CAD_HLA, LASTRTRV, SSL_REQUIRED, OFFSET, GUID, STG_HINT, UTILITYURL, KEEP_MP, TOCENTRIES, REPL_ARRULE, DECOMM_STATE, NODETYPE, ROLE_OVER, STATE, SPLIT_LARGE_OBJECTS, ARCHDEL, URL, PASSEXP, SESSIONINIT, 

In [33]:
df.count()

50482938

In [ ]:
df.select(['OBJID','NODEID', 'NODETYPE']).show()

In [24]:
# merge = merge.withColumn('POOLID_cast', merge.POOLID.cast(IntegerType()))
# merge = merge.withColumn('CHUNKID_cast', merge.CHUNKID.cast(LongType()))

In [25]:
cloud_chunkid = SD_CHUNK_LOCATIONS.where(SD_CHUNK_LOCATIONS.POOLID.isin(cloud)).select(SD_CHUNK_LOCATIONS.CHUNKID)
directory_chunkid = SD_CHUNK_LOCATIONS.where(SD_CHUNK_LOCATIONS.POOLID.isin(directory)).select(SD_CHUNK_LOCATIONS.CHUNKID)

In [40]:
directory_col = merge.select('CHUNKID').intersect(directory_chunkid)

In [42]:
cloud_col = merge.select('CHUNKID').intersect(cloud_chunkid)

In [44]:
merge = merge.withColumn('Output', F.when(merge.POOLID.isNotNull(), 0).when(merge.CHUNKID.isin(directory_col.CHUNKID), 1).when(merge.CHUNKID.isin(cloud_col.CHUNKID), 2))

In [46]:
df = merge
df = df.filter(df.Output.isNotNull())


In [48]:
df.select('Output').show()

+------+
|Output|
+------+
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
+------+
only showing top 20 rows



In [ ]:
# %%timeit
cloud_chunkid = []

for poolid in cloud:
    rows = SD_CHUNK_LOCATIONS.select(SD_CHUNK_LOCATIONS.CHUNKID).filter(F.when(SD_CHUNK_LOCATIONS.POOLID == poolid, True).otherwise(False)).distinct().collect()
    cloud_chunkid.extend([row['CHUNKID'] for row in rows])

In [ ]:
SD_CHUNK_LOCATIONS.where(SD_CHUNK_LOCATIONS.POOLID.isin(cloud)).select(SD_CHUNK_LOCATIONS.CHUNKID)

In [ ]:
cloud_chunkid = SD_CHUNK_LOCATIONS.where(SD_CHUNK_LOCATIONS.POOLID.isin(cloud)).select(SD_CHUNK_LOCATIONS.CHUNKID)

In [ ]:
directory_chunkid = SD_CHUNK_LOCATIONS.where(SD_CHUNK_LOCATIONS.POOLID.isin(directory)).select(SD_CHUNK_LOCATIONS.CHUNKID)

In [ ]:
type(directory_chunkid)

In [ ]:
directory_chunkid = directory_chunkid.withColumn('CHUNKID', directory_chunkid.CHUNKID.cast(LongType()))

In [ ]:
output_column = F.when(merge.POOLID.isNotNull(), 0).when(merge.CHUNKID.isin(directory_chunkid.CHUNKID), 1).when(merge.CHUNKID.isin(cloud_chunkid.CHUNKID), 2)

In [ ]:
merge_directory = merge.CHUNKID.isin(directory_chunkid.CHUNKID)

In [ ]:
merge_poolid

In [ ]:
merge.withColumn('Output', F.when(merge.POOLID.isNotNull(), 0).when(merge_directory, 1))

In [ ]:
df_merge = merge.withColumn('Output', F.when(merge.POOLID.isNotNull(), 0).when(merge.CHUNKID.isin(directory_chunkid.CHUNKID), 1).when(merge.CHUNKID.isin(cloud_chunkid.CHUNKID), 2))

In [ ]:
df_merge = merge.withColumn('OUTPUT', output_column)

In [ ]:
# %%timeit
directory_chunkid = []

for poolid in directory:
    rows = SD_CHUNK_LOCATIONS.select(SD_CHUNK_LOCATIONS.CHUNKID).filter(F.when(SD_CHUNK_LOCATIONS.POOLID == poolid, True).otherwise(False)).distinct().collect()
    directory_chunkid.extend([row['CHUNKID'] for row in rows])

In [ ]:
from pyspark.sql.functions import udf
def assign_output(poolid, chunkid):
    if poolid:
        return 0
    if chunkid in directory_chunkid:
        return 1
    elif chunkid:
        return 2
    else:
        return None

In [ ]:
output = udf(assign_output, IntegerType())

In [ ]:
%time
merge.select('OBJID').show(100)

In [ ]:
%time
output_column = output(merge.POOLID, merge.CHUNKID)

In [ ]:
df_merge = merge.withColumn('OUTPUT', output_column)

In [ ]:
%%time
df_merge = merge.withColumn('OUTPUT', output(merge.POOLID, merge.CHUNKID))

In [ ]:
df_merge.columns

In [ ]:
df_merge.select('OBJID','OUTPUT').show(1000)

In [ ]:
df_merge = df_merge.filter(df_merge.OUTPUT. isNotNull())

In [ ]:
%time
df_merge.select('OBJID').show(100)

In [ ]:
%time
df_merge.select('OUTPUT').show(100)

In [ ]:
%time
df_merge.select('OUTPUT')

In [ ]:
df_merge.count()

In [ ]:
df_merge.select('OUTPUT').distinct().show()

In [ ]:
df_merge.select('OUTPUT').count()

In [ ]:
directory_chunkid

In [ ]:
df = SDRO
df = df.filter(df.CHUNKID. isNotNull())
df = df.withColumn("CHUNKID", df.CHUNKID.cast(LongType()))
df.show(10)

In [ ]:
AFBF_POOLID = BACKUP_OBJECTS.join(AFBF, BACKUP_OBJECTS['OBJID'] == AFBF['BFID'], how='left').select('POOLID')

In [ ]:
SDRO_POOLID = BACKUP_OBJECTS.join(SDRO, ['OBJID'], how='left').select('POOLID')

In [ ]:
SDRO.columns

In [ ]:
[row['POOLID'] for row in merge.select("POOLID").distinct().collect()]

In [ ]:
df.groupby('POOLID').apply(mapY)

In [ ]:
directory = set([row['POOLID'] for row in AFBF.select("POOLID").distinct().collect()])
tape

In [ ]:
SD_CONTAINERS.columns

In [ ]:
# SD_CONTAINERS.select('POOLID', 'TYPE').show(100)
li2 = SD_CONTAINERS.select("POOLID").rdd.flatMap(lambda x: x).collect()
# set(li)

In [ ]:
set(li1).intersection(set(li2))

In [ ]:
set(li1).union(set(li2))

In [ ]:
merge = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])

for key, value in CSV_Merge_List.items():
    print(key, value)
    temp = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/" + key + ".csv"])
    if len(value) == 2:
            merge = merge.join(temp, merge[value[0]] == temp[value[1]], how='left')
    else:        
        merge = merge.join(temp, [value], how='left')

In [ ]:
df = merge
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("float"))
df.count()

In [ ]:
# full_outer_join = BACKUP_OBJECTS
# full_outer_join = full_outer_join.join(NODES, ['NODEID'], how='left')
# full_outer_join = full_outer_join.join(FILESPACES, ['FSID'], how='left')
# full_outer_join = full_outer_join.join(AFBF, full_outer_join.OBJID == AFBF.BFID, how='left')
# full_outer_join = full_outer_join.join(SDRO, ['OBJID'], how='left')
# full_outer_join = full_outer_join.join(SS_POOLS, ['POOLID'], how='left')
# full_outer_join = full_outer_join.join(SD_CHUNK_LOCATIONS, ['POOLID'], how='left')

In [ ]:
# df = full_outer_join
# df = df.filter(df.POOLID. isNotNull())
# df = df.withColumn("POOLID", df["POOLID"].cast("int"))
# df = df.filter(df.ATTRLENGTH. isNotNull())
# df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
# df = df.filter(df.BFSIZE. isNotNull())
# df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
# df = df.filter(df.HDRSIZE. isNotNull())
# df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
# df = df.filter(df.OBJID. isNotNull())
# df = df.withColumn("OBJID", df["OBJID"].cast("float"))

In [ ]:
df.select("OBJID", "ATTRLENGTH", "BFSIZE", "HDRSIZE", "POOLID").write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/merge_data/4_features")

In [ ]:
df.count()

In [ ]:
new_df = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"//merge_data/4_features/*.csv"])

In [ ]:
new_df.count()

In [ ]:
sc.stop()

In [11]:
# data exploration
import pandas as pd

In [5]:
data_dir = '/home/cole/Workspace/School/Capstone/data/first_data_set/TestData/'

In [13]:
nodes = pd.read_csv(data_dir+"/NODES.csv")
# backup_objects = pd.read_csv(data_dir+"/BACKUP_OBJECTS.csv")

In [ ]:
# backup_objects['nodeid']

In [7]:
backup_objects['NODEID']

0            12
1            12
2            12
3            12
4            12
5            12
6            12
7            12
8            12
9            12
10           12
11           12
12           12
13           12
14           12
15           12
16           12
17           12
18           12
19           12
20           12
21           12
22           12
23           12
24           12
25           12
26           12
27           12
28           12
29           12
           ... 
5823835    8597
5823836    8597
5823837    8597
5823838    8597
5823839    8597
5823840    8597
5823841    8597
5823842    8597
5823843    8597
5823844    8597
5823845    8597
5823846    8597
5823847    8597
5823848    8597
5823849    8597
5823850    8597
5823851    8597
5823852    8597
5823853    8597
5823854    8597
5823855    8597
5823856    8597
5823857    8597
5823858    8597
5823859    8597
5823860    8597
5823861    8597
5823862    8597
5823863    8597
5823864    8597
Name: NODEID, Length: 58

In [8]:
print(nodes.columns[0:50])

Index(['NODENAME', 'NODEID', 'PLATFORM', 'DOMAINNAME', 'PASSWORD', 'PWSETDATE',
       'CONTACT', 'CODEPAGE', 'COMPRESS', 'ENCRYPT', 'ARCHDEL', 'BACKDEL',
       'DISBACKDEL', 'LOCKSTATE', 'LACDATE', 'REGDATE', 'REGISTRAR',
       'NODESTATE', 'SESSCOMMMETH', 'SESSRECVD', 'SESSSENT', 'SESSDURATION',
       'SESSIDLEWAIT', 'SESSCOMMWAIT', 'SESSMEDIAWAIT', 'PWENCODED', 'PWRESET',
       'INVPWCOUNT', 'NODETYPE', 'CLNTVERSION', 'CLNTRELEASE', 'CLNTLEVEL',
       'CLNTSUBLEVEL', 'CLNTOSLEVEL', 'OPTIONSET', 'AGGREGATE', 'AGGR_ESTSIZE',
       'SESSCANCELED', 'URL', 'SERVERKEY', 'LFILES_STORED', 'PFILES_STORED',
       'BYTES_STORED', 'MAXNUMMP', 'DELTASTORED', 'FSRENAME', 'CRCREQUIRED',
       'TXNGROUPMAX', 'RESERVED1', 'RESERVED2'],
      dtype='object')


In [9]:
print(nodes.columns[50:])

Index(['OSNAME', 'PROCARCH', 'TGTVERSION', 'TGTRELEASE', 'TGTLEVEL',
       'TGTSUBLEVEL', 'CLNTPRODUCTS', 'AUTHENTICATION', 'SSL_REQUIRED',
       'PASSEXP', 'KEEP_MP', 'GUID_HLA', 'GUID_LLA', 'GUID', 'GUID_FORMAT',
       'DATA_WRITE_PATH', 'DATA_READ_PATH', 'SESSIONINIT', 'CLIENT_HLA',
       'CLIENT_LLA', 'DEDUP', 'BACKUPINIT', 'ROLE_OVER', 'ROLE', 'PVENDOR',
       'PBRAND', 'PTYPE', 'PMODEL', 'PCOUNT', 'HYPERVISOR', 'PAPI', 'PERROR',
       'MACADDR', 'REPL_STATE', 'REPL_MODE', 'REPL_BKRULE', 'REPL_ARRULE',
       'REPL_SPRULE', 'FAILOVER_WRT_DELAY', 'CAD_HLA', 'CAD_LLA',
       'CAD_ADDRTYPE', 'ANYV2CLIENT', 'APPVERSION', 'APPRELEASE', 'APPLEVEL',
       'APPSUBLEVEL', 'SPLIT_LARGE_OBJECTS', 'ATRISK_TYPE', 'ATRISK_INTERVAL',
       'UTILITYURL', 'RECOVER_DAMAGED', 'DECOMM_STATE', 'DECOMM_DATE',
       'ATRISK_TIMESTAMP', 'ATRISK_VALUE', 'S3_ACCESS_ID', 'S3_CREDGENDATE',
       'REPL_MODEPREV'],
      dtype='object')


In [14]:
nodes['OSNAME']

0                                                     NaN
1                                                     NaN
2                              WIN:Windows Server 2012 R2
3                                                     NaN
4             LNX:SUSE Linux Enterprise Server 11 (ppc64)
5                     LNX:SUSE Linux Enterprise Server 12
6       Red Hat Enterprise Linux Server release 6.4 (S...
7                     LNX:SUSE Linux Enterprise Server 12
8                     LNX:SUSE Linux Enterprise Server 12
9                     LNX:SUSE Linux Enterprise Server 12
10                                                    NaN
11      Red Hat Enterprise Linux Server release 6.4 (S...
12            LNX:SUSE Linux Enterprise Server 11 (ppc64)
13      Red Hat Enterprise Linux Server release 6.4 (S...
14            LNX:SUSE Linux Enterprise Server 11 (ppc64)
15                    LNX:SUSE Linux Enterprise Server 12
16            LNX:SUSE Linux Enterprise Server 11 (ppc64)
17      Red Ha

In [15]:
nodes['NODEID']

0          2
1          9
2         10
3         11
4         12
5         13
6         14
7         15
8         16
9         17
10      1681
11        20
12        21
13        22
14        23
15        25
16        26
17        27
18        28
19        29
20      1282
21      1283
22        18
23        19
24      1689
25      1331
26        34
27      1323
28      1330
29      1329
        ... 
3414    8288
3415    8289
3416    8290
3417    8291
3418    8292
3419    8293
3420    8294
3421    8295
3422    8296
3423    8297
3424    8298
3425    8299
3426    8300
3427    8301
3428    8302
3429    8303
3430    8304
3431    8305
3432    8306
3433    8307
3434    8308
3435    8309
3436    8310
3437    8311
3438    8312
3439    8313
3440    8314
3441    8315
3442    8316
3443    8317
Name: NODEID, Length: 3444, dtype: int64

In [16]:
nodes['NODESTATE']

0       0
1       0
2       5
3       0
4       3
5       3
6       3
7       3
8       3
9       3
10      0
11      3
12      3
13      3
14      3
15      3
16      3
17      3
18      3
19      3
20      3
21      3
22      3
23      3
24      3
25      0
26      0
27      0
28      0
29      0
       ..
3414    3
3415    3
3416    3
3417    3
3418    3
3419    3
3420    3
3421    3
3422    3
3423    3
3424    3
3425    3
3426    3
3427    3
3428    3
3429    3
3430    3
3431    3
3432    3
3433    3
3434    3
3435    3
3436    3
3437    3
3438    3
3439    3
3440    3
3441    3
3442    3
3443    5
Name: NODESTATE, Length: 3444, dtype: int64